In [2]:
from math import cos, sin, asin, acos, gcd, atan2
import numpy as np
from numpy import array, dot, degrees, cross
from numpy.linalg import inv, det, solve, norm
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# import the python scripts:
import csl_generator as csl
import gb_generator as gbc

%matplotlib notebook

In [4]:
help(csl.get_cubic_theta)

Help on function get_cubic_theta in module csl_generator:

get_cubic_theta(uvw, m, n=1)
    CSL analytical formula based on the book: 'Interfaces in crystalline materials',
     Sutton and Balluffi, clarendon press, 1996.



## Produce Lists of CSL boundaries for any given rotation axis (hkl) :



In [ ]:
axis = np.array([1, 2, 1])

# list Sigma boundaries < 50
csl.print_list(axis,50)


In [ ]:
csl.get_cubic_sigma

## Select a sigma and get the characteristics of the GB:

In [ ]:
# pick a sigma for this axis, ex: 15.
sigma = 15 

theta, m, n = csl.get_theta_m_n_list(axis,sigma)[0]

R = csl.rot(axis, theta)

# Minimal CSL cells. The plane orientations and the orthogonal cells
# will be produced from these original cells.
M1, M2 = csl.Create_minimal_cell_Method_1(sigma,axis,R)


print('Angle:',degrees(theta),'\n','Sigma:',sigma,'\n', 'Minimal cells:','\n', M1,'\n',M2,'\n', )

## Produce Lists of GB planes for the chosen boundary :


In [ ]:
# the higher the limit the higher the indices of GB planes produced.
lim = 8

V1, V2, M, Gb = csl.Create_Possible_GB_Plane_List(axis, m,n,lim)

df = pd.DataFrame(
    {'GB1': list(V1),
     'GB2': list(V2),
     'Type': Gb
    })

df.head()

## Criteria for finding the GB plane of ineterest*: 

###  1- Based on the type of GB plane: 

#### _* The following criteria searches the generated data frame. To extend the search you can increase the limit (lim)_
#### _in the above cell._

In [ ]:
# Gb types: Symmetric Tilt, Tilt, Twist, Mixed

df[df['Type'] == 'Mixed'].head()


### 2 - Based on the minimum number of atoms in the orthogonal cell:
####  _This can be of interest for DFT calculations that require smaller cells. The search may take a few minutes if your original limit_
#### _is large as it must calculate all the orthogonal cells to know the number of atoms._


In [ ]:
basis = 'fcc'
Number = np.zeros(len(V1))
for i in range((len(V1))):
    Number[i] = csl.Find_Orthogonal_cell(basis,axis,m,n,V1[i])[2]

In [ ]:
df['Number'] = list(Number)    

df[df['Number'] < 300]

### 3- Based on proximity to a particular boundary, for example the Symmetric tilts:
####  _This is how I created various steps on grain boundaries with vicinal orientations in the following work:_
#### __(https://journals.aps.org/prmaterials/abstract/10.1103/PhysRevMaterials.2.043601)__

In [ ]:
SymmTiltGbs = []
for i in range(len(V1)):
    if str(Gb[i]) == 'Symmetric Tilt':
        SymmTiltGbs.append(V1[i])

# Find GBs less than 5 degrees from any of the symmetric tilt boundaries in this system

Delta = 6
Min_angles = []
for i in range(len(V1)):
    angles = []
    for j in range(len(SymmTiltGbs)):
        angles.append(csl.angv(V1[i],SymmTiltGbs[j]))
    Min_angles.append(min(angles))    


In [ ]:
df['Angles'] = Min_angles

df[df['Angles'] < Delta]

## Select a GB plane and go on:
### You only need to pick the GB1 plane, from any of the three criteria in the cells above

In [ ]:
GB_plane = [-17, 8, 4]
# lattice parameter
LatP = 4
basis = 'fcc'

In [ ]:
# just a piece of info, how much of this mixed boundary is tilt or twist?
csl.Tilt_Twist_comp??

In [ ]:
# instantiate a GB:
my_gb = gbc.GB_character()

# give all the characteristics
my_gb.ParseGB (axis, basis, LatP, m, n, GB_plane)

# Create the bicrystal
my_gb.CSL_Bicrystal_Atom_generator()

# remove atoms closer than this fraction of the latice parameter. If put to 0 no atoms will be removed.
# if greate than 0 you must immediately input either g1 (grain 1) or g2 (grain 2) from which atoms will 
# be removed.

my_gb.overD = 0.3

# cell dimension [x,y,z], with x along GB normal and y and z in the GB plane

dim = [2,1,1]

# put in the rigib body translation to False: it shows the as constructed GB after atom removal, put it to True: 
# you must input two integers a and b to create a grid. by default a= 10 and b= 5 and you will produce 50 initial
# structures. For mor info, read the README.

#several possible ex:
    
# my_gb.WriteGBT(my_gb.overD,'g1', False, dim[0], dim[1], dim[2])
# my_gb.WriteGBT(0, True, 10, 5, dim[0], dim[1], dim[2])
# my_gb.WriteGB(0, False ,dim[0], dim[1], dim[2])

my_gb.WriteGB(my_gb.overD,'g2', False ,dim[0], dim[1], dim[2])


In [ ]:
## extract atom positions of the two grains:

X = my_gb.atoms1
Y = my_gb.atoms2

In [ ]:
# 3d plot of the gb, that can be shown in any cartesian view direction:

def plot_gb(X,Y, view_dir = [0,1,0]):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X[:,0],X[:,1],X[:,2],'o', s = 20, facecolor = 'y',edgecolor='none', alpha=0.2 )
    ax.scatter(Y[:,0],Y[:,1],Y[:,2],'o',s = 20,facecolor = 'b',edgecolor='none', alpha=0.2)
    
    # Show [0, 0, 0] as a red point
    ax.scatter(0,0,0,'s',s = 200,facecolor = 'r',edgecolor='none')
    ax.set_proj_type('ortho')
    ax.grid(False)
    az = degrees(atan2(view_dir[1],view_dir[0]))
    el = degrees(asin(view_dir[2]/norm(view_dir)))
    ax.view_init(azim = az, elev = el)       
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    return            


In [ ]:
%matplotlib notebook
plot_gb(X,Y,[0,1,0])